# Start a Project 


# Run Server

# Create App

# Django Views & URLs

#views.py Example:

>```python
from django.http import HttpResponse
def index(request):
    return HttpResponse("Hello, world. You're at the polls index.")
#urls.py Example:  

>```python
from django.urls import path
from . import views
urlpatterns = [
    path('', views.index, name='index'),
]

>```python
#main urls.py Example:
from django.contrib import admin
from django.urls import include, path
urlpatterns = [
    path('endpoint1/', include('app-name1.urls')),
    path('admin/', admin.site.urls),
]
#The include() function allows referencing other URLconfs
```

# Django Models


#appname/models.py example

>```python
from django.db import models
class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')
    # how the objects are displayed in backend
    def __str__(self):
        return self.question_text + self.pub_date
        
>```python
class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)
```

# Create SuperUser/Admin

### Make the app modifiable in the admin

## Extra about views & urls

### urls.py

>```python
urlpatterns =[
    # ex: /polls/
    path('', views.index, name='index'),
    # ex: /polls/5/
    path('<int:question_id>/', views.detail, name='detail'),
    path('<int:question_id>/results/', views.results, name='results'),
    ...
]

### views.py
>```python
from django.shortcuts import render
def detail(request, question_id):
    return HttpResponse(f"You're looking at question {question_id}"")
def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(f"{response} {question_id}")  
    context = {'key': 'value'}
    return render(request, 'appname/htmlfile.html', context)
 ```


# Change Django Admin Site

### Go to main_project.urls.py
>```python
admin.site.site_header = "UMSRA Admin"
admin.site.site_title = "UMSRA Admin Portal"
admin.site.index_title = "Welcome to UMSRA Researcher Portal"



## Important & Useful Tips:

1. Use Blocks & Extends in Jinja
2. Use generic views

# Django Rest Framework

>```python
pip install djangorestframework
##### after installing djangorestframework, add <mark>'rest_framework'</mark> to INSTALLED_APPS

>```python
#first import Response Object & api_view decorator
from django_restframework.response import Response
from django_restframework.decorator import api_view



>```python
@api_view(['GET'])
def getData(request):
    person={'name':'Rafiq'}
    return Response(person)
```


# Django Serializers

- > We need to create Serializers, because the Response Object can't natively handle <b><mark>Complex data type such as Django model instances</mark><b>

- > Serializers allow complex data such as querysets and model instances to be converted
to native Python datatypes that can then be easily rendered into JSON, XML or other content types.

- > Serializers also provide deserialization, allowing parsed data to be converted back into complex types,after first validating the incoming data

### Create a Simple Serializer

> ``` python
from rest_framework import serializers,JSONRenderer
class CommentSerializer(serializers.Serializer):
    email = serializers.EmailField()
    content = serializers.CharField(max_length=200)
    created = serializers.DateTimeField()
```

### Create Simple Object for Serialization

>```python
from datetime import datetime
class Comment:
    def __init__(self, email, content, created=None):
        self.email = email
        self.content = content
        self.created = created or datetime.now()
comment = Comment(email='leila@example.com', content='foo bar')
```

### Serializing Object

>```python
serializer = CommentSerializer(comment)
serializer.data
#{'email': 'leila@example.com', 'content': 'foo bar', 'created': '2016-01-27T15:17:10.375877'}
from rest_framework.renderers import JSONRenderer
json = JSONRenderer().render(serializer.data)
#b'{"email":"leila@example.com","content":"foo bar","created":"2016-01-27T15:17:10.375877"}'
```

## Deserializing objects
- Deserialization is similar. First we parse a stream into Python native datatypes...

>```python
import io
from rest_framework.parsers import JSONParser
stream = io.BytesIO(json)
data = JSONParser().parse(stream)


- then we restore those native datatypes into a dictionary of validated data.

>```python
serializer = CommentSerializer(data=data)
serializer.is_valid()
#True
serializer.validated_data
#{'content': 'foo bar', 'email': 'leila@example.com', 'created': datetime.datetime(2012, 08, 22, 16, 20, 09, 822243)}


# Using Serializers with Models

- If we want to use Serializer for any model which stores, password then we have to update the create method
of the serializer else the password will be stored as ir is in the database.

 ```python
from django_restframework import serializers
from appname.models import MyModel
```

```python
class MySerializer(serializers.ModelSerializer):
    class Meta:
        model = MyModel
        fields='__all__'
        # fields = ['field1', 'field2',...]
        extra_kwargs={'password':{'write_only':True}}
    def create(self, validated_data):
        mymodel = MyModel(
            email=validated_data['email'],
            username=validated_data['username'],
            gender= validated_data['dob']
        )
        mymodel.set_password(validated_data['password'])
        mymodel.save()
        return mymodel
```




## Django Use Serializer with the rest api

```python
from django_restframework import Response
from django_restframework.decorators import api_view
from appname.models import MyModel
from appname.serializers import MySerializer
```

```python
@api_view(['GET'])
def getData(request):
    objs=MyModel.Objects.all()
    serializer=MySerializer(objs,many=True)
    return Response(serializer.data)
    
    
@api_view(['POST'])
def addData(request):
    serializer=MySerializer(data=request.data)
    if serializer.is_valid():
        # if valid data, save object
        serializer.save()
        return Response(serializer.data,status=status.HTTP_201_CREATED)
    return Response(serializer.errors,status=status.HTTP_400_BAD_REQUEST)
```

## Viewset

> - Django REST framework allows us to combine the logic for a set of related views in a single class, called a ViewSet.

> - A ViewSet class is simply a type of class-based View, that does not provide any method handlers such as .get() or .post(), and instead provides actions such as .list() and .create()

> - we will register the viewset with a router, and allow the urlconf to be automatically generated.

 ```python
from myapp.views import MyViewSet
from rest_framework.routers import DefaultRouter

router = DefaultRouter()
#register the viewsets
router.register(r'url_prefix', MyViewSet, basename='MyViewSet')
urlpatterns = [path('',include(router.urls)), ...]
# urlpatterns += path.urls
```

## Create Viewset

### Use Inbuilt viewset which will create the basic tasks itself like: get,put
```python
class MyViewSet(viewsets.ModelViewSet):
    # define a queryset & Serializer
    queryset = MyModel.objects.all()
    serializer_class = MySerializer  
    # overrite create, to show custom message
    def create(self,request,*args,**kwargs):
        response=super().create(request,args,kwargs)
        if response.status_code == 201:
            return Response({'message':'User Created'},status=201)
        else:
            return response
        
        

# Django Knox Authentication

> pip install django-rest-knox

> INSTALLED_APPS = (
  ...
  'rest_framework',
  'knox',
  ...
)

>REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': ('knox.auth.TokenAuthentication',),
    ...
}


>```python
from rest_framework.permissions import IsAuthenticated
from rest_framework.response import Response
from rest_framework.views import APIView
from knox.auth import TokenAuthentication
from knox.views import LoginView as KnoxLoginView
from rest_framework.authtoken.serializers import AuthTokenSerializer
from django.contrib.auth import login
from rest_framework import permissions
class LoginView(KnoxLoginView):
    permission_classes = (permissions.AllowAny,)
    def post(self, request, format=None):
        serializer = AuthTokenSerializer(data=request.data)
        serializer.is_valid(raise_exception=True)
        user = serializer.validated_data['user']
        login(request, user)
        return super(LoginView, self).post(request, format=None)
```

## AWS S3 with django

### Change Settings.py
- AWS_ACCESS_KEY_ID =os.environ.get('AWS_ACCESS_KEY_ID')
- AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
- AWS_STORAGE_BUCKET_NAME=os.environ.get('AWS_STORAGE_BUCKET_NAME')

- AWS_S3_FILE_OVERWRITE = False
- AWS_DEFAULT_ACL=None
- DEFAULT_FILE_STORAGE= 'storages.backends.s3boto3.S3Boto3Storage' 
<br> </br>

- > Install 'django-storages','boto3' and add ''storages' to settings.py 
<br> </br>
 >To allow django-admin collectstatic to automatically put the static files in your bucket set the following in settings.py:
<br> </br>
 > STATICFILES_STORAGE = 'storages.backends.s3boto3.S3StaticStorage'


- models.FileField(upload_to=folder_name_s3)